# Import statements and reading in Nursery dataset into pandas DataFrame
- All features in the dataset are categorical
- No null values within the dataset

In [1]:
# Import statements
import pandas as pd
import numpy as np
from pprint import pprint

# Load and read nursery data set into pandas dataframe 
nurseryDF = pd.read_csv("nursery.data", names=["parents", "has_nurs", "form", "children", 
                                                "housing", "finance", "social", "health",
                                                "label"])

# Print information about dataframe
nurseryDF.info()
display(nurseryDF)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12960 entries, 0 to 12959
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   parents   12960 non-null  object
 1   has_nurs  12960 non-null  object
 2   form      12960 non-null  object
 3   children  12960 non-null  object
 4   housing   12960 non-null  object
 5   finance   12960 non-null  object
 6   social    12960 non-null  object
 7   health    12960 non-null  object
 8   label     12960 non-null  object
dtypes: object(9)
memory usage: 911.4+ KB


,parents,has_nurs,form,children,housing,finance,social,health,label
0,usual,proper,complete,1,convenient,convenient,nonprob,recommended,recommend
1,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
2,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
3,usual,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend
4,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority
...,...,...,...,...,...,...,...,...,...
12955,great_pret,very_crit,foster,more,critical,inconv,slightly_prob,priority,spec_prior
12956,great_pret,very_crit,foster,more,critical,inconv,slightly_prob,not_recom,not_recom
12957,great_pret,very_crit,foster,more,critical,inconv,problematic,recommended,spec_prior
12958,great_pret,very_crit,foster,more,critical,inconv,problematic,priority,spec_prior


# Splitting data into training, testing and post-pruning sets
- Training (60%)
- Testing (20%)
- Post-pruning (20%)

In [2]:
# Function to split the data into training, testing and post-pruning sets
def train_test_prune_split(df, train_size, test_size):
    train_size = round(train_size * len(df))
    test_size = round(test_size * len(df))
    post_prune_size = len(df) - train_size - test_size
    
    # Using random sampling to split the data
    np.random.seed(10)
    shuffled_indices = np.random.permutation(len(df))

    train_indices = shuffled_indices[:train_size]
    test_indices = shuffled_indices[train_size:train_size + test_size]
    post_prune_indices = shuffled_indices[train_size + test_size:]

    train_df = df.iloc[train_indices]
    test_df = df.iloc[test_indices]
    prune_df = df.iloc[post_prune_indices]

    return train_df, test_df, prune_df

In [3]:
# Calling the function to split the dataset
train_df, test_df, prune_df = train_test_prune_split(nurseryDF, 0.6, 0.2)

# Helper functions for building the decision tree

In [4]:
# Function to check if the data is pure
def check_purity(data):
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    # If no. of unique class is only 1 (out of possuble 5), the data is pure
    if len(unique_classes) == 1:
        return True
    else:
        return False

# Function to classify the data (return the class that has the highest no. of occurence within the dataset)
def classify_data(data):
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

# Function to get all the potential splits while building the decision tree
def get_potential_splits(data):
    potential_splits = {}
    _, n_columns = data.shape

    # Get unique values for each feature
    for column_index in range(n_columns - 1): # excludes the last column which is the label
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)
                
        potential_splits[column_index] = unique_values
    
    return potential_splits

# Function to split the data into left and right datasets given a split column and value
def split_data(data, split_column, split_value):
    split_column_values = data[:, split_column]

    # Check if column value is equal to split value, store in left dataset if true else store in right dataset
    data_left = data[split_column_values == split_value]
    data_right = data[split_column_values !=  split_value]
    
    return data_left, data_right

# Splitting criteria by Entropy(Information Gain): Helper functions
- Lowest entropy for a split will result in the highest information gain

In [5]:
# Function to calculate entropy
def calculate_entropy(data):
    
    # Get the label values
    label_column = data[:, -1]

    # Count the number of each unique value
    _, unique_counts = np.unique(label_column, return_counts=True)

    # Calculate the probability of each column values
    probabilities = unique_counts / unique_counts.sum()

    # Compute and return entropy
    return sum(probabilities * -np.log2(probabilities))

# Function to calculate overall entropy from split
def calculate_overall_entropy(data_left, data_right):

    n = len(data_left) + len(data_right)
    p_data_left = len(data_left) / n
    p_data_right = len(data_right) / n

    overall_entropy =  (p_data_left * calculate_entropy(data_left) + p_data_right * calculate_entropy(data_right))
    
    return overall_entropy

# Function to determine the best feature and value to be the splitting node through information gain
def determine_best_split_IG(data, potential_splits):
    
    # Instantiate a variable to store the value of the lowest entropy while looping through the potential splits
    lowest_entropy = 99999

    # Loop through the potential splits (both by column and value)
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            # Call helper functions to split the data and then calculate overall entropy
            data_left, data_right = split_data(data, column_index, value)
            overall_entropy = calculate_overall_entropy(data_left, data_right)

            # If the overall entropy of this split is lower than the current lowest entropy,
            # set the computed entropy as the new lowest and set the new best splitting criteria
            if overall_entropy <= lowest_entropy:
                lowest_entropy = overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value



# Splitting criteria by Gain Ratio: Helper functions

In [6]:
# Function to determine the best feature and value to be the splitting node through gain ratio
def determine_best_split_GR(data, potential_splits):
    
    # Instantiate a variable to store the value of the highest gain ratio
    highest_gain_ratio = -1

    # Calculate the total entropy of the current data
    dataset_entropy = calculate_entropy(data)
    
    # Loop through the potential splits (both by column and value)
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            # Call helper functions to split the data and then calculate overall entropy
            data_left, data_right = split_data(data, column_index, value)
            overall_entropy = calculate_overall_entropy(data_left, data_right)
            gain_ratio = (dataset_entropy - overall_entropy) / overall_entropy

            # If the gain ratio of this split is higher than the current highest gain ratio,
            # set the computed gain ratio as the new highest and set the new best splitting criteria
            if gain_ratio >= highest_gain_ratio:
                highest_gain_ratio = gain_ratio
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value


# Splitting criteria by Gini Index: Helper functions
- For each split, the lower the gini index/impurity, the better the split as a lot more of the data can be differentiated

In [7]:
def calculate_gini_index(left_data, right_data):
    
    # Get the two column values
    col_1 = left_data[:,-1]
    col_2 = right_data[:,-1]
    
    # Count each unique values for each column values
    _, num_counts_1 = np.unique(col_1, return_counts=True)
    _, num_counts_2 = np.unique(col_2, return_counts=True)
    
    # Calculate the probability of each column values
    prob_1 = num_counts_1 / num_counts_1.sum()
    prob_2 = num_counts_2 / num_counts_2.sum()
    
    # Calculate Gini of left and right dataset
    gini_left = 1 - sum(prob_1**2)
    gini_right = 1 - sum(prob_2**2)
    
    # Total number of rows in left and right dataset
    total_length = len(left_data) + len(right_data)

    # Calculate the gini index of the split
    gini_index = (len(left_data)/total_length) * gini_left + (len(right_data)/total_length) * gini_right
    
    return gini_index

# Function to determine the best feature and value to be the splitting node through gini index
def determine_best_split_GI(data, potential_splits):
    
    # Instantiate a variable to store the value of the lowest gini index while looping through the potential splits
    lowest_GI = 99999
    
    # Loop through the potential splits (both by column and value)
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            # Call helper functions to split the data and then calculate gini index for that split
            data_left, data_right = split_data(data, column_index, value)
            current_GI = calculate_gini_index(data_left, data_right)

            # If the gini index of this split is lower than the current lowest gini index,
            # set the computed gini index as the new lowest and set the new best splitting criteria
            if current_GI <= lowest_GI:
                lowest_GI = current_GI
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

# Tree induction algorithm
Important parameters:
- A maximum depth parameter prevents the tree from growing too large and causing overfitting (we use a default of 10 to begin with)
- Splitting criteria determines the splitting criteria out of the 3(Information gain, Gain Ratio, Gini Index) to be used when building the tree

In [8]:
def build_tree(df, counter=0, max_depth=10, splitting_criteria="gini_index"):
    # Instantiating variables if starting at the root
    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df           
    
    
    # Check if the data is impure or if the tree has already reached the specified max depth
    # If true, returns the classification without splitting
    if (check_purity(data)) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification
    
    # Splitting algorithm
    else:
        # Increase counter by one and call helper function to get the potential splits among the features
        counter += 1
        potential_splits = get_potential_splits(data)

        # Use the method of splitting specified
        if (splitting_criteria == "info_gain"):
            split_column, split_value = determine_best_split_IG(data, potential_splits)
        elif (splitting_criteria == "gain_ratio"):
            split_column, split_value = determine_best_split_GR(data, potential_splits)
        else:
            split_column, split_value = determine_best_split_GI(data, potential_splits)
        
        data_left, data_right = split_data(data, split_column, split_value)
        
        # Build the tree
        feature_name = COLUMN_HEADERS[split_column]
        question = "{} = {}".format(feature_name, split_value)
        tree = {question: []}
        
        # Perform recursion to traverse the new left and right nodes of the tree
        yes_answer = build_tree(data_left, counter, max_depth, splitting_criteria)
        no_answer = build_tree(data_right, counter, max_depth, splitting_criteria)
        
        # For cases when data is classified even though it is not pure yet (e.g. tree has reached max depth)
        if yes_answer == no_answer:
            tree = yes_answer
        else:
            tree[question].append(yes_answer)
            tree[question].append(no_answer)
        
        return tree


# Classfication function (prediction)

In [9]:
def classify_example(test_data, tree):
    # Dictionary that stores all the nodes and it's values]
    if isinstance(tree, str):
        return tree
    else:
        question = list(tree.keys())[0]
    name_of_feature, operator, val = question.split(" ")

    if str(test_data[name_of_feature]) == val:
        ans = tree[question][0]
    else:
        ans = tree[question][1]

    # Check if selected node is a dictionary (base case)
    if not isinstance(ans, dict):
        return ans
    
    # If tree is a dictionary, call classify function using test data and remaining tree (recursive part)
    else:
        remaining_tree = ans
        return classify_example(test_data, remaining_tree)

# Prediciton function based on decision tree
def predict(test_data, tree):
    result = []
    idx = test_data.index.tolist()
    for i in idx:
        result.append(classify_example(test_data.loc[i],tree))
    return result

# Evaluation of models
- Test built decision tree with testing dataset
- Compute the mean accuracy of all 3 models

## Building tree based on Information Gain splitting criteria

In [10]:
# Build the tree based on splitting criteria = "info_gain"
IG_tree = build_tree(train_df, splitting_criteria="info_gain")

# Make predictions based off of testing dataset
final_prediction_IG = predict(test_df, IG_tree)

# Check accuracy of model by comparing actual values with predicted values by calculating the mean of the two values
accuracy_IG = (final_prediction_IG == test_df["label"]).mean()

# Display accuracy
print('Accuracy of Information Gain tree =', accuracy_IG)


Accuracy of Information Gain tree = 0.9645061728395061


## Building tree based on Gain Ratio splitting criteria

In [11]:
# Build the tree based on splitting criteria = "gain_ratio"
GR_tree = build_tree(train_df, splitting_criteria="gain_ratio")

# Make predictions based off of testing dataset
final_prediction_GR = predict(test_df, GR_tree)

# Check accuracy of model by comparing actual values with predicted values and calculating the mean of the two values
accuracy_GR = (final_prediction_GR == test_df["label"]).mean()

# Display accuracy
print('Accuracy of Gain Ratio tree =', accuracy_GR)

<ipython-input-6-4cdfe6cab148>:16: RuntimeWarning: divide by zero encountered in double_scalars
  gain_ratio = (dataset_entropy - overall_entropy) / overall_entropy


Accuracy of Gain Ratio tree = 0.9645061728395061


## Building tree based on Gini Index splitting criteria

In [12]:
# Build the tree based on splitting criteria = "gini_index"
GI_tree = build_tree(train_df, splitting_criteria="gini_index")

# Make predictions based off of testing dataset
final_prediction_GI = predict(test_df, GI_tree)

# Check accuracy of model by comparing actual values with predicted values and calculating the mean of the two values
accuracy_GI = (final_prediction_GI == test_df["label"]).mean()

# Display accuracy
print('Accuracy of Gini Index tree =', accuracy_GI)

Accuracy of Gini Index tree = 0.9672067901234568


# Post-pruning helper functions
- For each devision node in the tree, determine if we should keep the node or replace it with a node

In [13]:
def filter_df(df, question):
    feature_name, operator, val = question.split()
    
    df_yes = df[df[feature_name].astype(str) == val]
    df_no  = df[df[feature_name].astype(str) != val]
    
    return df_yes, df_no

def determine_leaf(df_train):
    return df_train.label.value_counts().index[0]

def determine_errors(df_val, tree):
    final_predictions = predict(df_val, tree)
    actual_val = df_val.label
    
    return sum(final_predictions != actual_val)

def pruning_result(tree, dataframe_train, dataframe_val):
    
    leaf_node = determine_leaf(dataframe_train)
    leaf_errors = determine_errors(dataframe_val, leaf_node)
    errors_decision_node = determine_errors(dataframe_val, tree)

    if leaf_errors <= errors_decision_node:
        return leaf_node
    else:
        return tree

def post_pruning(tree, df_train, df_val):
    
    question = list(tree.keys())[0]
    yes_answer, no_answer = tree[question]

    # base case
    if not isinstance(yes_answer, dict) and not isinstance(no_answer, dict):
        return pruning_result(tree, df_train, df_val)
        
    # recursion to prune the tree
    else:
        df_train_yes, df_train_no = filter_df(df_train, question)
        df_val_yes, df_val_no = filter_df(df_val, question)
        
        if isinstance(yes_answer, dict):
            yes_answer = post_pruning(yes_answer, df_train_yes, df_val_yes)
            
        if isinstance(no_answer, dict):
            no_answer = post_pruning(no_answer, df_train_no, df_val_no)
        
        tree = {question: [yes_answer, no_answer]}
    
        return pruning_result(tree, df_train, df_val)

# Implementing post-pruning when building the trees and determining the best max_depth parameter to prevent overfitting

## Post-pruning Information Gain decision tree
- The metrics show that a maximum depth of 14 seem to be the most optimal for building the tree

In [14]:
metrics = {"max_depth": [], "acc_tree": [], "acc_tree_pruned": []}

# Testing range of max_depth parameter from 5 to 25
for n in range (5, 26):
    IG_tree = build_tree(train_df, max_depth=n, splitting_criteria="info_gain")
    IG_tree_pruned = post_pruning(IG_tree, train_df, prune_df)

    metrics["max_depth"].append(n)

    final_predictions_IG = predict(test_df, IG_tree)
    metrics["acc_tree"].append((final_prediction_IG == test_df["label"]).mean())

    final_prediction_IG_pruned = predict(test_df, IG_tree_pruned)
    metrics["acc_tree_pruned"].append((final_prediction_IG_pruned == test_df["label"]).mean())

df_metrics = pd.DataFrame(metrics)
df_metrics = df_metrics.set_index("max_depth")
display(df_metrics)

,acc_tree,acc_tree_pruned
max_depth,,
5,0.964506,0.869985
6,0.964506,0.892361
7,0.964506,0.912423
8,0.964506,0.940201
9,0.964506,0.946373
10,0.964506,0.962577
11,0.964506,0.974151
12,0.964506,0.979552
13,0.964506,0.985725


## Post-pruning Gain Ratio decision tree
- - The metrics show that a maximum depth of 14 seem to be the most optimal for building the tree

In [15]:
metrics = {"max_depth": [], "acc_tree": [], "acc_tree_pruned": []}

# Testing range of max_depth parameter from 5 to 25
for n in range (5, 26):
    GR_tree = build_tree(train_df, max_depth=n, splitting_criteria="gain_ratio")
    GR_tree_pruned = post_pruning(GR_tree, train_df, prune_df)

    metrics["max_depth"].append(n)

    final_predictions_GR = predict(test_df, GR_tree)
    metrics["acc_tree"].append((final_prediction_GR == test_df["label"]).mean())

    final_prediction_GR_pruned = predict(test_df, GR_tree_pruned)
    metrics["acc_tree_pruned"].append((final_prediction_GR_pruned == test_df["label"]).mean())

df_metrics = pd.DataFrame(metrics)
df_metrics = df_metrics.set_index("max_depth")
display(df_metrics)

<ipython-input-6-4cdfe6cab148>:16: RuntimeWarning: divide by zero encountered in double_scalars
  gain_ratio = (dataset_entropy - overall_entropy) / overall_entropy


,acc_tree,acc_tree_pruned
max_depth,,
5,0.964506,0.869985
6,0.964506,0.892361
7,0.964506,0.912423
8,0.964506,0.940201
9,0.964506,0.946373
10,0.964506,0.962577
11,0.964506,0.974151
12,0.964506,0.979552
13,0.964506,0.985725


## Post-pruning Gini Index decision tree
- - The metrics show that a maximum depth of 13 seem to be the most optimal for building the tree

In [16]:
metrics = {"max_depth": [], "acc_tree": [], "acc_tree_pruned": []}

# Testing range of max_depth parameter from 5 to 25
for n in range (5, 26):
    GI_tree = build_tree(train_df, max_depth=n, splitting_criteria="gini_index")
    GI_tree_pruned = post_pruning(GI_tree, train_df, prune_df)

    metrics["max_depth"].append(n)

    final_predictions_GI = predict(test_df, GI_tree)
    metrics["acc_tree"].append((final_prediction_GI == test_df["label"]).mean())

    final_prediction_GI_pruned = predict(test_df, GI_tree_pruned)
    metrics["acc_tree_pruned"].append((final_prediction_GI_pruned == test_df["label"]).mean())

df_metrics = pd.DataFrame(metrics)
df_metrics = df_metrics.set_index("max_depth")
display(df_metrics)

,acc_tree,acc_tree_pruned
max_depth,,
5,0.967207,0.875772
6,0.967207,0.896991
7,0.967207,0.915509
8,0.967207,0.940972
9,0.967207,0.950617
10,0.967207,0.964506
11,0.967207,0.975694
12,0.967207,0.980710
13,0.967207,0.986497
